# Random Forest 401

## Librerías y carga de datos

In [1]:
#from google.colab import drive
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import os
import time
import pickle

C:\Users\Diego\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Diego\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Diego\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


Se monta drive en caso de que se trabaje en Colab. En otro caso, comentar celda:

In [2]:
#drive.mount('/content/drive')

In [3]:
path_train = 'train2.csv'
path_test = 'test2.csv'
path_val = 'val2.csv'

train=pd.read_csv(path_train, delimiter = ',')

## Modelo Random Forest

#### Se crea columna de diferencia de distancia entre dos pulsos pareados.

In [4]:
train['Distancia_Total'] = train['DistanciaInicio2'] - train['DistanciaInicio1']

In [5]:
with open('encoding1.pickle', 'rb') as handle:
    servs_encoding = pickle.load(handle)

In [6]:
train['Servicio'] = train['Servicio'].apply(lambda x: servs_encoding[x])

#### Creación del modelo y definición de hiperparámetros
En este caso se entrena con la columna de distancia como única feature y con el target la columna de diferencia de tiempo.

In [7]:
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
rf.fit(train[['Distancia_Total', 'Servicio','dia_habil1','dia_Monday1','dia_Saturday1','dia_Sunday1','dia_Thursday1',
                 'dia_Tuesday1','dia_Wednesday1']], train[['Diferencia']])

C:\Users\Diego\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [26]:
#se guarda el modelo entrenado para usarlo más adelante
rf_path = 'RandomForest401.sav'
pickle.dump(rf, open(rf_path, 'wb'))

In [8]:
train=[] #liberamos memoria

#### Validación

In [13]:
val=pd.read_csv(path_val, delimiter = ',') #se carga conjunto de validación

In [14]:
val['Distancia_Total'] = val['DistanciaInicio2'] - val['DistanciaInicio1'] #columna utilizada para predecir

In [15]:
val['Servicio'] = val['Servicio'].apply(lambda x: servs_encoding[x])

In [24]:
def eval_range(model, dataset, top=60, bottom=0):
    """función que retorna error de la predicción por rango de tiempo"""  
    condition1 = dataset["Diferencia"] >= bottom
    condition2 = dataset["Diferencia"] < top
    total_cond = np.logical_and(condition1, condition2)
    filtered = dataset[total_cond]
    if filtered.shape[0] > 0:
        y_val = filtered['Diferencia']
        X_val = filtered[['Distancia_Total', 'Servicio','dia_habil1','dia_Monday1','dia_Saturday1','dia_Sunday1','dia_Thursday1',
                 'dia_Tuesday1','dia_Wednesday1']].values
        y_pred = model.predict(X_val)
        rms = mean_squared_error(y_val, y_pred)
        mae = mean_absolute_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        dif = y_val - y_pred
        me = np.sum(dif)/(len(y_pred))
        mpe = np.sum(dif[dif > 0])/(len(y_pred))
        return rms, mae, me, mpe, r2
    else:
        return None, None, None

In [25]:
rms, mae, me, mpe, r2 = eval_range(rf, val[['Distancia_Total', 'Servicio','dia_habil1','dia_Monday1','dia_Saturday1','dia_Sunday1','dia_Thursday1',
                 'dia_Tuesday1','dia_Wednesday1','Diferencia']], top=1e20, bottom=0)
print("Errores globales")
print("\tRMS: \t{}".format(rms))
print("\tMAE: \t{}".format(mae))
print("\tME: \t{}".format(me))
print("\tMPE: \t{}".format(mpe))
print("\tR2: \t{}\n".format(r2))


ranges = [[0, 5*60], [5*60, 10*60], [10*60, 20*60], [20*60, 40*60], [40*60, 60*60]]
for bottom, top in ranges:
    rms, mae, me, mpe, r2 = eval_range(rf, val[['Distancia_Total', 'Servicio','dia_habil1','dia_Monday1','dia_Saturday1','dia_Sunday1','dia_Thursday1',
                 'dia_Tuesday1','dia_Wednesday1', 'Diferencia']], top=top, bottom=bottom)
    print("Errores para el rango {}min a {}min".format(bottom//60, top//60))
    print("\tRMS: \t{}".format(rms))
    print("\tMAE: \t{}".format(mae))
    print("\tME: \t{}".format(me))
    print("\tMPE: \t{}".format(mpe))
    print("\tR2: \t{}\n".format(r2))

Errores globales
	RMS: 	94788.17250631074
	MAE: 	182.5345473332528
	ME: 	-8.07147774339763
	MPE: 	87.23153479492758
	R2: 	0.8640279648596396

Errores para el rango 0min a 5min
	RMS: 	6025.456820578285
	MAE: 	50.2295250348247
	ME: 	-29.946178868349822
	MPE: 	10.141673083237439
	R2: 	-0.04291924636188238

Errores para el rango 5min a 10min
	RMS: 	34157.01168811552
	MAE: 	132.18109354354004
	ME: 	-50.8919354202952
	MPE: 	40.64457906162241
	R2: 	-3.6157135168339725

Errores para el rango 10min a 20min
	RMS: 	111542.17615512761
	MAE: 	241.91863204027518
	ME: 	-72.8718954397152
	MPE: 	84.52336830028
	R2: 	-2.7707538741138587

Errores para el rango 20min a 40min
	RMS: 	236525.32025696742
	MAE: 	378.7852391754356
	ME: 	-19.610515449711766
	MPE: 	179.58736186286188
	R2: 	-1.0229942980373252

Errores para el rango 40min a 60min
	RMS: 	424232.9788081344
	MAE: 	521.914432815181
	ME: 	413.6322502987734
	MPE: 	467.77334155697724
	R2: 	-2.6004369606296294

